In [1]:
import json
import logging
import sys
import warnings
from pathlib import Path

import ipytest

%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

logging.getLogger('sagemaker').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [3]:
import sagemaker
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/kmitura/.config/sagemaker/config.yaml


In [4]:
import os
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

DUMMY_ROLE = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-11111111111111"
BUCKET = os.environ["BUCKET"]
S3_LOCATION = f"s3://{BUCKET}/football"
LOCAL_MODE = False

role = os.environ["ROLE"]

architecture = !(uname -m)
IS_ARM64_ARCHITECTURE = architecture[0] == "arm64"

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=BUCKET),
        "instance_type": "local",
        "image": "sagemaker-xgboost-training-toolkit-local" if IS_ARM64_ARCHITECTURE else None
    }
else:
    config = {
        "session": PipelineSession(default_bucket=BUCKET) if not LOCAL_MODE else None,
        "instance_type": "ml.c5.xlarge",
        "image": None,
    }

config["framework_version"] = "1.7-1"
config["py_version"] = "py310"

In [5]:


df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

# S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])
# S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])

In [6]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [7]:
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.sklearn import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.retry import (
    SageMakerJobExceptionTypeEnum,
)

retry_policy = {
    "ExceptionType": SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
    "IntervalSeconds": 2,
    "BackoffRate": 2,
    "MaxAttempts": 5,
    "ExpireAfterMin": 1
}

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

processor = SKLearnProcessor(
    base_job_name="split-and-transform-data",
    framework_version="1.2-1",
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config['session'],
)

split_and_transform_data_step = ProcessingStep(
    name="split-and-transform-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),
        ],
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),
            ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline"),
            ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline"),
        ]
    ),
    cache_config=cache_config,
)

In [8]:
# from sagemaker.workflow.pipeline import Pipeline
# 
# session1_pipeline = Pipeline(
#     name="session1-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config['session'],
# )
# 
# session1_pipeline.upsert(role)
# # session1_pipeline.start()

In [9]:
from sagemaker.xgboost import XGBoost

use_spot_instances = True and not LOCAL_MODE
max_run = 1000
max_wait = 1200 if use_spot_instances else None
instance_type = "ml.m5.2xlarge"

xgboost = XGBoost(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    role=role,
    instance_count=1,
    instance_type=config['instance_type'],
    framework_version=config['framework_version'],
    disable_profiler=True,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    sagemaker_session=config['session']
)

In [10]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput


def create_training_step(estimator):
    """Create a SageMaker TrainingStep using the provided estimator."""
    return TrainingStep(
        name="train-model",
        step_args=estimator.fit(
            inputs={
                "train": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "train"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "validation"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
            },
        ),
        cache_config=cache_config,
    )

train_model_step = create_training_step(xgboost)

In [11]:
USE_TUNING_STEP = False and not LOCAL_MODE

In [12]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'eta': ContinuousParameter(min_value=0.05, max_value=0.3, scaling_type="Logarithmic"),
    'max_depth': IntegerParameter(min_value=5, max_value=15, scaling_type="Auto"),
    'subsample': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Auto"),
    'colsample_bytree': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(min_value=5, max_value=12, scaling_type="Logarithmic"),
    'alpha': ContinuousParameter(min_value=1, max_value=10, scaling_type="Logarithmic"),
    'min_child_weight': ContinuousParameter(min_value=0.4, max_value=1.0, scaling_type="Auto"),
}

objective_type = "Maximize"
metric_definitions = [
    {'Name': 'validation:logloss',
     'Regex': r".*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*"
     },
    {'Name': 'validation:f1',
     'Regex': 'F1 score: ([0-9\\.]+)'
     },
]
metric_name = "validation:f1"
strategy = "Bayesian"

tuner = HyperparameterTuner(
    estimator=xgboost,
    objective_metric_name=metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    early_stopping_type='Auto',
)

In [13]:
from sagemaker.workflow.steps import TuningStep

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        },
    ),
    cache_config=cache_config
)

In [14]:
# session2_pipeline = Pipeline(
#     name="session2-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session2_pipeline.upsert(role_arn=role)
# # session2_pipeline.start()

In [15]:
from sagemaker.xgboost import XGBoostProcessor

evaluation_processor = XGBoostProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

In [16]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [17]:
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

if USE_TUNING_STEP:
    model_assets = tune_model_step.get_top_model_s3_uri(
        top_k=0,
        s3_bucket=config["session"].default_bucket(),
    )

In [18]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            ProcessingInput(
                source=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

In [19]:
# session3_pipeline = Pipeline(
#     name="session3-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#         evaluate_model_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session3_pipeline.upsert(role_arn=role)
# # session3_pipeline.start()

In [20]:
BASIC_MODEL_PACKAGE_GROUP = "basic-football"

In [21]:
from sagemaker.xgboost.model import XGBoostModel

xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

In [22]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.functions import Join

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs[
                    "evaluation"
                ].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    ),
)

In [23]:
from sagemaker.workflow.model_step import ModelStep

def create_registration_step(
        model,
        model_package_group_name,
        approval_status="Approved",
        content_types=["text/csv"],
        response_types=["text/csv"],
        model_metrics=None,
        drift_check_baselines=None,
):
    """Create a Registration Step using the supplied parameters."""
    return ModelStep(
        name="register",
        step_args=model.register(
            model_package_group_name=model_package_group_name,
            approval_status=approval_status,
            model_metrics=model_metrics,
            drift_check_baselines=drift_check_baselines,
            content_types=content_types,
            response_types=response_types,
            inference_instances=[config["instance_type"]],
            transform_instances=[config["instance_type"]],
            framework_version=config["framework_version"],
            domain="MACHINE_LEARNING",
            task="CLASSIFICATION",
            framework="XGBOOST",
        ),
    )


register_model_step = create_registration_step(
    xgb_model,
    BASIC_MODEL_PACKAGE_GROUP,
    model_metrics=model_metrics,
)

In [24]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo

f1_threshold = ParameterFloat(name="f1_threshold", default_value=0.64)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's f1 result was lower than",
            f1_threshold,
        ],
    ),
)

condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.f1.value",
    ),
    right=f1_threshold,
)

condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [25]:
# session5_pipeline = Pipeline(
#     name="session5-pipeline",
#     parameters=[dataset_location, f1_threshold],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#         evaluate_model_step,
#         condition_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session5_pipeline.upsert(role_arn=role)
# # session5_pipeline.start()

In [26]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=BASIC_MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = (
    response["ModelPackageSummaryList"][0]
    if response["ModelPackageSummaryList"]
    else None
)

# package

In [27]:
lambda_role_name = "lambda-deployment-role"
lambda_role_arn = None

try:
    response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": ["lambda.amazonaws.com", "events.amazonaws.com"],
                        },
                        "Action": "sts:AssumeRole",
                    },
                ],
            },
        ),
        Description="Lambda Endpoint Deployment",
    )

    lambda_role_arn = response["Role"]["Arn"]

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
        RoleName=lambda_role_name,
    )

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
        RoleName=lambda_role_name,
    )

    print(f'Role "{lambda_role_name}" created with ARN "{lambda_role_arn}".')
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = response["Role"]["Arn"]
    print(f'Role "{lambda_role_name}" already exists with ARN "{lambda_role_arn}".')

Role "lambda-deployment-role" already exists with ARN "arn:aws:iam::284415450706:role/lambda-deployment-role".


In [28]:
from sagemaker.lambda_helper import Lambda

ENDPOINT = "football-endpoint"
DATA_CAPTURE_DESTINATION = f"{S3_LOCATION}/monitoring/data-capture"
DATA_CAPTURE_PERCENTAGE = 100
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline-football"

deploy_lambda_fn = Lambda(
    function_name="deployment_fn",
    execution_role_arn=lambda_role_arn,
    script=(CODE_FOLDER / "lambda" / "lambda.py").as_posix(),
    handler="lambda.lambda_handler",
    timeout=600,
    session=sagemaker_session,
    runtime="python3.11",
    environment={
        "Variables": {
            "ENDPOINT": ENDPOINT,
            "DATA_CAPTURE_DESTINATION": DATA_CAPTURE_DESTINATION,
            "DATA_CAPTURE_PERCENTAGE": str(DATA_CAPTURE_PERCENTAGE),
            "ROLE": role,
            "MODEL_PACKAGE_GROUP": PIPELINE_MODEL_PACKAGE_GROUP
        },
    },
)

deploy_lambda_fn_response = deploy_lambda_fn.upsert()
deploy_lambda_fn_response

{'ResponseMetadata': {'RequestId': 'c6938d7b-ee00-4a8a-b233-71265adc2527',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 08 May 2024 09:05:51 GMT',
   'content-type': 'application/json',
   'content-length': '1648',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c6938d7b-ee00-4a8a-b233-71265adc2527'},
  'RetryAttempts': 0},
 'FunctionName': 'deployment_fn',
 'FunctionArn': 'arn:aws:lambda:eu-north-1:284415450706:function:deployment_fn',
 'Runtime': 'python3.11',
 'Role': 'arn:aws:iam::284415450706:role/lambda-deployment-role',
 'Handler': 'lambda.lambda_handler',
 'CodeSize': 3151,
 'Description': '',
 'Timeout': 600,
 'MemorySize': 128,
 'LastModified': '2024-05-08T09:05:51.000+0000',
 'CodeSha256': 'ZwWvJBgXZu7zZcM11xwaUnLdUwdGUA0Xk8wc2aXySQ8=',
 'Version': '$LATEST',
 'Environment': {'Variables': {'ROLE': 'arn:aws:iam::284415450706:role/service-role/AmazonSageMaker-ExecutionRole-20240309T101533',
   'DATA_CAPTURE_PERCENTAGE': '100',
   'DATA_CAPTURE_DESTINATION': 

In [29]:
from sagemaker.workflow.lambda_step import LambdaStep


def create_deployment_step(register_model_step):
    """Create a Deploy Step using the supplied parameters."""
    return LambdaStep(
        name="deploy",
        lambda_func=deploy_lambda_fn,
        inputs={
            "model_package_arn": register_model_step.properties.ModelPackageArn,
        },
    )


deploy_step = create_deployment_step(register_model_step)

In [30]:
condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step, deploy_step],
    else_steps=[fail_step],
)

In [31]:
# session6_pipeline = Pipeline(
#     name="session6-pipeline",
#     parameters=[dataset_location, f1_threshold],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#         evaluate_model_step,
#         condition_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session6_pipeline.upsert(role_arn=role)

In [32]:
# session6_pipeline.start()

In [33]:
PENDING_MODEL_PACKAGE_GROUP = "pending-football"

In [34]:
event_pattern = f"""
{{
  "source": ["aws.sagemaker"],
  "detail-type": ["SageMaker Model Package State Change"],
  "detail": {{
    "ModelPackageGroupName": ["{PENDING_MODEL_PACKAGE_GROUP}"],
    "ModelApprovalStatus": ["Approved"]
  }}
}}
"""

In [35]:
rule_name = "PendingModelApprovedRule"

events_client = boto3.client("events")
rule_response = events_client.put_rule(
    Name=rule_name,
    EventPattern=event_pattern,
    State="ENABLED",
    RoleArn=role,
)

response = events_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            "Id": "1",
            "Arn": deploy_lambda_fn_response["FunctionArn"],
        },
    ],
)

lambda_function_name = deploy_lambda_fn_response["FunctionName"]
lambda_client = boto3.client("lambda")

try:
    response = lambda_client.add_permission(
        Action="lambda:InvokeFunction",
        FunctionName=lambda_function_name,
        Principal="events.amazonaws.com",
        SourceArn=rule_response["RuleArn"],
        StatementId="EventBridge",
    )
except lambda_client.exceptions.ResourceConflictException:
    print(f'Function "{lambda_function_name}" already has the specified permission.')

Function "deployment_fn" already has the specified permission.


In [36]:
register_model_step = create_registration_step(
    xgb_model,
    PENDING_MODEL_PACKAGE_GROUP,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
)

condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [37]:
# session7_pipeline = Pipeline(
#     name="session7-pipeline",
#     parameters=[dataset_location, f1_threshold],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#         evaluate_model_step,
#         condition_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session7_pipeline.upsert(role_arn=role)
# # session7_pipeline.start()

In [38]:
transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [39]:
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=role,
)
xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version='1.2-1',
    sagemaker_session=config["session"],
    role=role,
)

from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=role,
)

In [40]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline-football"

In [41]:
register_model_step = create_registration_step(
    pipeline_model,
    PIPELINE_MODEL_PACKAGE_GROUP,
    content_types=["text/csv", ],
    response_types=["text/csv", ],
    model_metrics=model_metrics,
)

deploy_step = create_deployment_step(register_model_step)

condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step, deploy_step],
    else_steps=[fail_step],
)

In [42]:
# session_pipeline = Pipeline(
#     name="session-pipeline",
#     parameters=[dataset_location, f1_threshold],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#         evaluate_model_step,
#         condition_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session_pipeline.upsert(role_arn=role)
# # session_pipeline.start()

In [43]:
from datetime import datetime

GROUND_TRUTH_LOCATION = f"{S3_LOCATION}/monitoring/groundtruth"
DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"
MODEL_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/model-quality"

In [44]:
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    QualityCheckStep,
)

data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=config["session"],
        role=role,
    ),
    quality_check_config=DataQualityCheckConfig(
        baseline_dataset=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "train-baseline"
        ].S3Output.S3Uri,
        dataset_format=DatasetFormat.csv(header=True),
        output_s3_uri=DATA_QUALITY_LOCATION,
    ),
    model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config,
)

In [45]:
from sagemaker.drift_check_baselines import DriftCheckBaselines

data_quality_model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

data_quality_drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
)

In [46]:
register_model_step = create_registration_step(
    pipeline_model,
    PIPELINE_MODEL_PACKAGE_GROUP,
    content_types=["text/csv", "application/json"],
    response_types=["text/csv", "application/json"],
    model_metrics=data_quality_model_metrics,
    drift_check_baselines=data_quality_drift_check_baselines,
)

In [47]:
condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [48]:
# session9_pipeline = Pipeline(
#     name="session9-pipeline",
#     parameters=[dataset_location, f1_threshold],
#     steps=[
#         split_and_transform_data_step,
#         train_model_step,
#         evaluate_model_step,
#         data_quality_baseline_step,
#         condition_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session9_pipeline.upsert(role_arn=role)

In [49]:
# session9_pipeline.start()

In [50]:
# from sagemaker.s3 import S3Downloader
# 
# try:
#     response = json.loads(
#         S3Downloader.read_file(f"{DATA_QUALITY_LOCATION}/statistics.json"),
#     )
#     print(json.dumps(response["features"][49], indent=2))
# except Exception:
#     pass

In [51]:
# try:
#     response = json.loads(
#         S3Downloader.read_file(f"{DATA_QUALITY_LOCATION}/constraints.json"),
#     )
#     print(json.dumps(response, indent=2))
# except Exception:
#     pass

In [52]:
create_model_step = ModelStep(
    name="create-model",
    step_args=pipeline_model.create(instance_type=config["instance_type"]),
)

In [53]:
from sagemaker.transformer import Transformer

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    instance_type=config["instance_type"],
    instance_count=1,
    strategy="MultiRecord",
    accept="text/csv",
    assemble_with="Line",
    output_path=f"{S3_LOCATION}/transform",
    sagemaker_session=config["session"],
)

In [54]:
from sagemaker.workflow.steps import TransformStep

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    step_args=transformer.transform(
        # We will use the baseline set we generated when we split the data.
        # This set corresponds to the test split before the transformation step.
        data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "test-baseline"
        ].S3Output.S3Uri,
        join_source="Input",
        split_type="Line",
        content_type="text/csv",
        # We want to output the first and the second to last field from
        # the joint set. The first field corresponds to the groundtruth,
        # and the second to last field corresponds to the prediction.
        #
        # Notice how the first field is the groundtruth coming from the
        # test set. The second to last field is the prediction coming the
        # model.
        output_filter="$[-3,-2]",
    ),
    cache_config=cache_config,
)

In [55]:
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",
    check_job_config=CheckJobConfig(
        instance_type="ml.m5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=config["session"],
        role=role,
    ),
    quality_check_config=ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,
        dataset_format=DatasetFormat.csv(header=False),
        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="MulticlassClassification",
        # Since the data doesn't have headers, SageMaker will autocreate headers for it.
        # _c0 corresponds to the first column, and _c1 corresponds to the second column.
        ground_truth_attribute="_c0",
        inference_attribute="_c1",
        output_s3_uri=MODEL_QUALITY_LOCATION,
        
    ),
    model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config,
)

In [56]:
from sagemaker.drift_check_baselines import DriftCheckBaselines

model_quality_model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

model_quality_drift_check_baselines = DriftCheckBaselines(
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
)

In [57]:
register_model_step = create_registration_step(
    pipeline_model,
    PIPELINE_MODEL_PACKAGE_GROUP,
    content_types=["text/csv", "application/json"],
    response_types=["text/csv", "application/json"],
    model_metrics=model_quality_model_metrics,
    drift_check_baselines=model_quality_drift_check_baselines,
)

In [58]:
condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=(
        [
            create_model_step,
            generate_test_predictions_step,
            model_quality_baseline_step,
            deploy_step
        ]
    ),
    else_steps=[fail_step],
)

In [59]:
from sagemaker.workflow.pipeline import Pipeline

session10_pipeline = Pipeline(
    name="session10-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        train_model_step,
        evaluate_model_step,
        data_quality_baseline_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session10_pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session10-pipeline',
 'ResponseMetadata': {'RequestId': '9d90dd9d-558a-4f2b-a7f6-d56b7d919fda',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9d90dd9d-558a-4f2b-a7f6-d56b7d919fda',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '87',
   'date': 'Wed, 08 May 2024 09:05:58 GMT'},
  'RetryAttempts': 0}}

In [60]:
# session10_pipeline.start()

In [61]:
from sagemaker.s3 import S3Downloader

try:
    response = json.loads(
        S3Downloader.read_file(f"{MODEL_QUALITY_LOCATION}/constraints.json"),
    )
    print(json.dumps(response, indent=2))
except Exception:
    pass

{
  "version": 0.0,
  "multiclass_classification_constraints": {
    "accuracy": {
      "threshold": 0.6030871003307607,
      "comparison_operator": "LessThanThreshold"
    },
    "weighted_recall": {
      "threshold": 0.6030871003307607,
      "comparison_operator": "LessThanThreshold"
    },
    "weighted_precision": {
      "threshold": 0.6371214166058752,
      "comparison_operator": "LessThanThreshold"
    },
    "weighted_f0_5": {
      "threshold": 0.608972579378614,
      "comparison_operator": "LessThanThreshold"
    },
    "weighted_f1": {
      "threshold": 0.5892193930857728,
      "comparison_operator": "LessThanThreshold"
    },
    "weighted_f2": {
      "threshold": 0.5909092539695012,
      "comparison_operator": "LessThanThreshold"
    }
  }
}


In [62]:
# from sagemaker import ModelPackage
# from sagemaker.model_monitor import DataCaptureConfig
# 
# 
# def deploy_model():
#     """Deploy the latest model registered in the Model Registry."""
#     response = sagemaker_client.list_model_packages(
#         ModelPackageGroupName=PIPELINE_MODEL_PACKAGE_GROUP,
#         ModelApprovalStatus="Approved",
#         SortBy="CreationTime",
#         MaxResults=1,
#     )
# 
#     package = (
#         response["ModelPackageSummaryList"][0]
#         if response["ModelPackageSummaryList"]
#         else None
#     )
# 
#     if package:
#         model_package = ModelPackage(
#             model_package_arn=package["ModelPackageArn"],
#             sagemaker_session=sagemaker_session,
#             role=role,
#         )
# 
#         model_package.deploy(
#             endpoint_name=ENDPOINT,
#             initial_instance_count=1,
#             instance_type=config["instance_type"],
#             # We must enable Data Capture to monitor the model.
#             data_capture_config=DataCaptureConfig(
#                 enable_capture=True,
#                 sampling_percentage=100,
#                 destination_s3_uri=DATA_CAPTURE_DESTINATION,
#                 capture_options=["REQUEST", "RESPONSE"],
#                 csv_content_types=["text/csv"],
#                 json_content_types=["application/json"],
#             ),
#         )
#         
# sagemaker_client.delete_endpoint_config(
#     EndpointConfigName='football-endpoint'
# )
# 
# deploy_model()

In [63]:
from sagemaker.base_serializers import CSVSerializer
from sagemaker.base_deserializers import CSVDeserializer, JSONDeserializer
from sagemaker import Predictor
import pandas as pd

def read_last_few_lines(csv_path):
    chunk_size = 2000
    last_chunk = None

    for chunk in pd.read_csv(csv_path, chunksize=chunk_size):
        last_chunk = chunk

    if last_chunk is not None:
        return last_chunk.tail(10)
    else:
        return pd.DataFrame()

df = read_last_few_lines(df_local_path)

In [64]:
predictor = Predictor(
    endpoint_name=ENDPOINT,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
    sagemaker_session=sagemaker_session,
)

# def generate_fake_traffic(data):
#     """Generate fake traffic to the endpoint."""
#     df_ = data.copy()
#     df_.reset_index(drop=True, inplace=True)
#     for index, row in df_.iterrows():
#         print(f'index: {index}')
#         payload = ",".join([str(x) for x in row.to_list()])
#         predictor.predict(
#             payload,
#             initial_args={"ContentType": "text/csv", "Accept": "text/csv"},
#             # The `inference_id` field is important to match
#             # it later with a corresponding ground-truth label.
#             inference_id=str(index),
#         )
# 
# generate_fake_traffic(df)

In [66]:
files = S3Downloader.list(DATA_CAPTURE_DESTINATION)
if len(files):
    lines = S3Downloader.read_file(files[-1])
    print(f"File: {files[-1]}")
    print(json.dumps(json.loads(lines.split("\n")[0]), indent=2))

File: s3://football-data-kamil/football/monitoring/data-capture/football-endpoint/AllTraffic/2024/05/07/10/49-52-573-0a495977-f20c-430b-b3f3-5900bda11891.jsonl
{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "72.0,72.0,72.0,72.0,72.0,72.0,70.0,71.0,75.0,77.0,76.0,75.0,75.0,75.0,75.0,75.0,75.0,60.0,79.0,67.0,74.0,76.0,1.31,1.99,2.51,0.77,44.0,38.0,1.9897295928105807,1.6952394889987854,72.82,73.27,21.74,14.01,5.33,4.913,51.094,53.04,73.5,71.83,75.0,72.0,72.81818181818181,73.27272727272727,0.0,0.0,13.3783,33.2579185520362,24.85467128027681,8.403247271759387",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "home_win,0.5370060205459595\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "4a856a8f-c39a-4161-b683-9e2fceb3a8d9",
    "inferenceId": "0",
    "inferenceTime": "2024-05-07T10:49:52Z"
  

In [67]:
import random
from datetime import datetime

from sagemaker.s3 import S3Uploader

records = []
for inference_id in range(len(df)):
    random.seed(inference_id)

    records.append(
        json.dumps(
            {
                "groundTruthData": {
                    "data": random.choice(["home_win", "home_not_win"]),
                    "encoding": "CSV",
                },
                "eventMetadata": {
                    "eventId": str(inference_id),
                },
                "eventVersion": "0",
            },
        ),
    )

groundtruth_payload = "\n".join(records)
upload_time = datetime.utcnow()
uri = f"{GROUND_TRUTH_LOCATION}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
S3Uploader.upload_string_as_file_body(groundtruth_payload, uri)

's3://football-data-kamil/football/monitoring/groundtruth/2024/05/08/09/1146.jsonl'

In [74]:
from sagemaker.model_monitor import ModelQualityMonitor

model_monitor = ModelQualityMonitor(
    instance_type=config["instance_type"],
    instance_count=1,
    max_runtime_in_seconds=1800,
    volume_size_in_gb=20,
    role=role,
    sagemaker_session=sagemaker_session
)

In [75]:
import time

from sagemaker.model_monitor import CronExpressionGenerator, EndpointInput

model_monitor.create_monitoring_schedule(
    monitor_schedule_name="football-model-monitoring-schedule",
    endpoint_input=EndpointInput(
        endpoint_name=predictor.endpoint_name,
        inference_attribute="0",
        destination="/opt/ml/processing/input_data",
    ),
    problem_type="MulticlassClassification",
    ground_truth_input=GROUND_TRUTH_LOCATION,
    constraints=f"{MODEL_QUALITY_LOCATION}/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    output_s3_uri=MODEL_QUALITY_LOCATION,
    enable_cloudwatch_metrics=True,
)

time.sleep(10)
model_monitor.start_monitoring_schedule()

ClientError: An error occurred (ValidationException) when calling the CreateModelQualityJobDefinition operation: Endpoint 'football-endpoint' does not exist or is not valid

In [ ]:
import json
from sagemaker.s3 import S3Downloader

def check_execution(monitoring_schedule):
    """Check the execution of the Monitoring Job.

    This function checks the execution of the Monitoring
    Job and prints out the list of violations if the job
    completed.
    """
    executions = monitoring_schedule.list_executions()

    if executions:
        execution = executions[-1].describe()
        print(f"Processing Job Status: {execution['ProcessingJobStatus']}")

        if execution["ProcessingJobStatus"] == "Completed":
            # print(f"Exit Message: \"{execution['ExitMessage']}\"")
            print(
                f"Last Modified Time: {execution['LastModifiedTime']}",
                end="\n\n",
            )
            print("Execution:")
            print(json.dumps(execution, default=str, indent=2), end="\n\n")

            latest_monitoring_violations = (
                monitoring_schedule.latest_monitoring_constraint_violations()
            )
            
            print(f'latest_monitoring_violations: {latest_monitoring_violations}')
            
            response = json.loads(
                S3Downloader.read_file(latest_monitoring_violations.file_s3_uri),
            )
            print("Violations:")
            print(json.dumps(response, indent=2))


In [76]:
check_execution(model_monitor)

ParamValidationError: Parameter validation failed:
Invalid type for parameter MonitoringScheduleName, value: None, type: <class 'NoneType'>, valid types: <class 'str'>

In [77]:
try:
    model_monitor.delete_monitoring_schedule()
except Exception as e:
    print(e)
    
# # Define the monitoring schedule name
# monitoring_schedule_name = "football-model-monitoring-schedule"
# 
# # Delete the monitoring schedule
# sagemaker_session.delete_monitoring_schedule(monitoring_schedule_name)

Parameter validation failed:
Invalid type for parameter MonitoringScheduleName, value: None, type: <class 'NoneType'>, valid types: <class 'str'>


In [90]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=BASIC_MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=2,
)

if response["ModelPackageSummaryList"]:
    production_package = response["ModelPackageSummaryList"][1]["ModelPackageArn"]
    shadow_package = response["ModelPackageSummaryList"][0]["ModelPackageArn"]
else:
    production_package = None
    shadow_package = None

print(f"Production package: {production_package}")
print(f"Shadow package: {shadow_package}")

Production package: arn:aws:sagemaker:eu-north-1:284415450706:model-package/basic-football/10
Shadow package: arn:aws:sagemaker:eu-north-1:284415450706:model-package/basic-football/11


In [91]:
SHADOW_DEPLOYMENT_ENDPOINT = "football-endpoint"
timestamp = time.strftime("%m%d%H%M%S", time.localtime())

In [94]:
production_model_name = f"{SHADOW_DEPLOYMENT_ENDPOINT}-production-{timestamp}"

sagemaker_client.create_model(
        ModelName=production_model_name,
        ExecutionRoleArn=role,
        PrimaryContainer={"ModelPackageName": production_package}
    )

In [ ]:
shadow_model_name = f"{SHADOW_DEPLOYMENT_ENDPOINT}-shadow-{timestamp}"

sagemaker_client.create_model(
    ModelName=shadow_model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"ModelPackageName": shadow_package}
)

In [82]:
# Retrieve model details
response = sagemaker_client.describe_model(ModelName=production_model_name)

# Print the container configuration
print(response['PrimaryContainer'])

{'Mode': 'SingleModel', 'ModelPackageName': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/pipeline-football/48'}


In [83]:
# Retrieve model details
response = sagemaker_client.describe_model(ModelName=shadow_model_name)

# Print the container configuration
print(response['PrimaryContainer'])

{'Mode': 'SingleModel', 'ModelPackageName': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/pipeline-football/49'}


In [78]:
SHADOW_DATA_DESTINATION = f"{S3_LOCATION}/endpoint/"

In [93]:
endpoint_config_name = f"{SHADOW_DEPLOYMENT_ENDPOINT}-config-{timestamp}"

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ModelName": production_model_name,
            "InstanceType": "ml.m5.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "VariantName": "ProductionTraffic",
        },
    ],
    ShadowProductionVariants=[
        {
            "ModelName": shadow_model_name,
            "InstanceType": "ml.m5.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "VariantName": "ShadowTraffic",
        },
    ],
    DataCaptureConfig={
        "EnableCapture": True,
        "InitialSamplingPercentage": 100,
        "DestinationS3Uri": SHADOW_DATA_DESTINATION,
        "CaptureOptions": [
            {"CaptureMode": "Input"},
            {"CaptureMode": "Output"},
        ],
        "CaptureContentTypeHeader": {
            "CsvContentTypes": ["text/csv", "application/octect-stream"],
            "JsonContentTypes": ["application/json", "application/octect-stream"],
        },
    },
)

In [ ]:
sagemaker_client.create_endpoint(
    EndpointName=SHADOW_DEPLOYMENT_ENDPOINT,
    EndpointConfigName=endpoint_config_name,
)